<a href="https://colab.research.google.com/github/koll-ai/control-meme-api/blob/main/control_meme_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title # Install dependencies

! pip install "git+https://github.com/takuma104/diffusers.git@controlnet" # Diffusers in development version
! pip install transformers accelerate safetensors xformers opencv-python

from diffusers import StableDiffusionControlNetPipeline, EulerAncestralDiscreteScheduler
from diffusers.utils import load_image
import torch
import cv2
import numpy as np
from PIL import Image

from flask import Flask
from flask import request
import base64
from io import BytesIO

import subprocess
import time

In [ ]:
#@title # Load ControlNet pipeline

model = "takuma104/control_sd15_canny" #@param {type:"string"}

#Common
euler_scheduler = EulerAncestralDiscreteScheduler.from_config("takuma104/control_sd15_canny", subfolder="scheduler")

#Canny Edge model
pipe_canny = StableDiffusionControlNetPipeline.from_pretrained("takuma104/control_sd15_canny", torch_dtype=torch.float16).to("cuda")
pipe_canny.scheduler = euler_scheduler
pipe_canny.enable_xformers_memory_efficient_attention()


In [ ]:
#@title # Setup localtunnel
with open('output.txt', 'w') as f:
    proc = subprocess.Popen(['lt', '--port', '5000'], stdout=f)

time.sleep(3)

In [ ]:
#@title # Run the service

app = Flask(__name__)

@app.route('/', methods=['POST'])
def index():
    params = request.get_json()

    generator = torch.Generator(device="gpu").manual_seed(params["seed"])

    image = pipe_canny(prompt=params["prompt"], 
                      negative_prompt=params["negative_prompt"],
                      controlnet_hint=params["controlnet_hint"],
                      num_inference_steps=params["num_inference_steps"], 
                      generator=generator).images[0]
    
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    
    return base64.b64encode(buffered.getvalue())

if __name__ == '__main__':
  with open('output.txt', 'r') as f:
      contents = f.read()
      API_URL = contents.split(' ')[-1]
      protocol = API_URL.split(':')[0]
      url = API_URL.split('/')[-1]

      print(API_URL)

      print('\n'*2)
      print(f'Everything is ready! Click on the this link to be redirected to koll.ai. Don\'t close this tab!')
      print(f'https://meme.koll.ai?protocol={protocol}&url={url}')

  app.run()